### **Import and Install Libraries**

In [ ]:
import os
import cv2
import copy
import pandas as pd
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
import random
import torch
from sklearn import svm
from skimage.morphology import disk
from sklearn.preprocessing import StandardScaler
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.functional import Tensor
from scipy.ndimage.filters import median_filter
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from scipy.stats import kurtosis, skew
import torchvision.transforms.functional as TF
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from torch.utils.data import Dataset, DataLoader
from torch.nn.modules.activation import ReLU
from sklearn.metrics import classification_report, balanced_accuracy_score
from sklearn.linear_model import LogisticRegression
from scipy.stats import kurtosis, skew
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedShuffleSplit
import seaborn as sns
import pickle
from torch.nn.modules.batchnorm import BatchNorm2d
import torch.optim as optim
from tqdm import tqdm
from google.colab import drive
from pathlib import Path
from sklearn.utils import shuffle


In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### **Preprocessing: Hair Removal**

In [ ]:
######## Create Rotating and Tilted Structuring Element ##########
def createTiltedStructuringElements(width, height, n):
  # width = 11
  # height = 1
  # n = 30
  base = np.zeros((width, width), np.uint8)
  start = int(width/2-height/2)
  end = int(width/2+height/2)
  for k in range(start+1, end+1):
    cv2.line(base,(0,k),(width,k),(255,255,255))
  SE_list = []
  SE_list.append(base)
  angle_step = 360.0/n
  for i in range(n):
    rows = base.shape[0]
    cols = base.shape[1]
    M = cv2.getRotationMatrix2D((cols/2.0, rows/2.0), i*angle_step, 1.0)
    SE = cv2.warpAffine(base, M, (width, width),  cv2.INTER_NEAREST);
    SE_list.append(SE)
  return SE_list

In [ ]:
## Creating Tilted Structuring Element for sum of TopHats
SE_list = []
SE_list = createTiltedStructuringElements(11, 1, 40)

In [ ]:
#Creating a function for removing the hairs
def remove_hair(src_image, SE_list):

  #Removing noise from the image
  median_image = median_filter(src_image, 3)
  #Converting to grayscale for appling gs morphology
  gray_image = cv2.cvtColor(median_image, cv2.COLOR_BGR2GRAY)
  #Inverting the intentisities of the pixels before using tophat transform
  inv_image = cv2.bitwise_not(gray_image)
  #Defining a matrix to store the sum of tophats
  sumOfTopHats = np.zeros((inv_image.shape[0], inv_image.shape[1]), np.uint16)

  #Applying a sum of tophat transform with different rotated SE to the images
  for SE in SE_list:
    tophat_image = cv2.morphologyEx(inv_image, cv2.MORPH_TOPHAT, SE)
    tophat_image = np.uint16(tophat_image)
    sumOfTopHats += tophat_image
  #Normalizing the intensities values of the pixels
  cv2.normalize(sumOfTopHats, sumOfTopHats, 0, 255, cv2.NORM_MINMAX);
  #Converting back to uint8 for visualization
  sumOfTopHats = np.uint8(sumOfTopHats)
  #Changing againg the pixels intensities before thresholding
  inv_tophat = cv2.bitwise_not(sumOfTopHats)
  #Thresholding the image to create a mask
  ret, thres_image = cv2.threshold(sumOfTopHats, 10, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

  #Applying morphological operations for removing remaining dark objects
  disk_kernel = disk(5)
  closed_image1 = cv2.morphologyEx(thres_image, cv2.MORPH_CLOSE, disk_kernel)
  # dil_kernel = disk(5)
  dil_kernel = np.ones((3,3),np.uint8)
  dilation_image = cv2.dilate(closed_image1,dil_kernel,iterations = 1)

  #Inpainting with the dilated mask for removing remaining dark objects
  inpainted_image = cv2.inpaint(src_image,dilation_image,20,cv2.INPAINT_TELEA)

  return inpainted_image

### **Preprocessing: Vignette Removal**

In [ ]:
#Defining a circular mask for the FOV removal algorithm
def create_circular_mask(h, w, center, radius):
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    return mask

In [ ]:
#Defining a function for FOV removal
def remove_black_border(inp_image):
  with_border_img = inp_image
  #Defining the size of the circular mask
  h, w = with_border_img.shape[:2]
  center = (int(w/2), int(h/2))
  radius = min(center[0], center[1], w-center[0], h-center[1])
  mask = create_circular_mask(h,w,center,radius)
  masked_img =  with_border_img.copy()
  #Counting the pixels intensities
  count_black = np.count_nonzero(masked_img[~mask] < 50)
  count_white = np.count_nonzero(masked_img[~mask] >= 50)
  count_total = count_black + count_white
  check = count_black/count_total*100
  diff = 100
  #Setting a condition for identifying whether there is a FOV in the image
  if(check<10):
    return inp_image
  else:
  #Counting the intensities values of the pixels and subtracting the circular mask from the original image
  #until there is no more black ring. 
    while diff>10:
      radius = radius-8 #The radius is reduced on each iteration to subtract remaining black pixels
      mask = create_circular_mask(h,w,center,radius)
      #Initial pixels intensities values on each iteration
      old_black = count_black
      old_white = count_white
      old_total = count_total
      #Counting the intensities values on each iteration
      count_black = np.count_nonzero(masked_img[~mask] < 50)
      count_white = np.count_nonzero(masked_img[~mask] >= 50)
      count_total = count_black + count_white
      #Checking if the stopping condition is met
      diff_black = abs(count_black-old_black)
      diff_white = abs(count_white-old_white)
      diff_total = abs(count_total-old_total)
      diff = diff_black/(diff_total+1)*100
  masked_img[~mask] = 255
  return  masked_img


### **Data Loading**

In [ ]:
# Directories
input_directory = '/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem'
pp_directory = '/content/gdrive/MyDrive/Colab Notebooks/CAD Project/Dataset/Three Class Problem Preprocessed'
train_directory = input_directory+'/train' 
val_directory = input_directory+'/val'
pp_train_directory = pp_directory+'/train' 
pp_val_directory = pp_directory+'/val'

In [ ]:
#Training set organization
train_bcc= sorted(glob(os.path.join(train_directory, "bcc", "*.jpg")))
train_mel= sorted(glob(os.path.join(train_directory, "mel", "*.jpg")))
train_scc= sorted(glob(os.path.join(train_directory, "scc", "*.jpg")))
tags_bcc_train=[]
tags_mel_train=[]
tags_scc_train=[]
tags_train=[]
images_bcc=[]
images_mel=[]
images_scc=[]
labels_bcc=[]
labels_mel=[]
labels_scc=[]

In [ ]:
#Validation set organization
train_bcc_val= sorted(glob(os.path.join(val_directory, "bcc", "*.jpg")))
train_mel_val= sorted(glob(os.path.join(val_directory, "mel", "*.jpg")))
train_scc_val= sorted(glob(os.path.join(val_directory, "scc", "*.jpg")))
tags_bcc_val=[]
tags_mel_val=[]
tags_scc_val=[]
tags_val=[]
images_bcc_val=[]
images_mel_val=[]
images_scc_val=[]
labels_bcc_val=[]
labels_mel_val=[]
labels_scc_val=[]

In [ ]:
print(len(train_mel))
print(len(train_bcc))
print(len(train_scc))
print(len(train_bcc_val))
print(len(train_mel_val))
print(len(train_scc_val))

2713
2713
2632
498
678
94


In [ ]:
dim = (512, 512)

**Training Dataframe Construction**

In [ ]:
for i in range(len(train_mel)):
  path_image = os.path.join(pp_train_directory, "mel", train_mel[i][-12:])
  image_read= cv2.imread(train_mel[i])
  image_resized =  cv2.resize(image_read, dim, interpolation = cv2.INTER_AREA)
  image_hair_removed= remove_hair(image_resized, SE_list)
  image_vignette_removed= remove_black_border(image_hair_removed)
  cv2.imwrite(path_image, image_vignette_removed)
  tags_mel_train.append(train_mel[i][-12:])
  labels_mel.append(0)

In [ ]:
for i in range(len(train_bcc)):
  path_image = os.path.join(pp_train_directory, "bcc", train_bcc[i][-12:])
  image_read= cv2.imread(train_bcc[i])
  image_resized =  cv2.resize(image_read, dim, interpolation = cv2.INTER_AREA)
  image_hair_removed= remove_hair(image_resized, SE_list)
  image_vignette_removed= remove_black_border(image_hair_removed)
  cv2.imwrite(path_image, image_vignette_removed)
  tags_bcc_train.append(train_bcc[i][-12:])
  labels_bcc.append(1)

In [ ]:
for i in range(len(train_scc)):
  path_image = os.path.join(pp_train_directory, "scc", train_scc[i][-12:])
  image_read= cv2.imread(train_scc[i])
  image_resized =  cv2.resize(image_read, dim, interpolation = cv2.INTER_AREA)
  image_hair_removed= remove_hair(image_resized, SE_list)
  image_vignette_removed= remove_black_border(image_hair_removed)
  cv2.imwrite(path_image, image_vignette_removed)
  tags_scc_train.append(train_scc[i][-12:])
  labels_scc.append(2)

**Validation Dataframe Creation**

In [ ]:
for i in range(len(train_mel_val)):
  path_image = os.path.join(pp_val_directory, "mel", train_mel_val[i][-12:])
  image_read= cv2.imread(train_mel_val[i])
  image_resized =  cv2.resize(image_read, dim, interpolation = cv2.INTER_AREA)
  image_hair_removed= remove_hair(image_resized, SE_list)
  image_vignette_removed= remove_black_border(image_hair_removed)
  cv2.imwrite(path_image, image_vignette_removed)
  tags_mel_val.append(train_mel_val[i][-12:])
  labels_mel_val.append(0)

In [ ]:
for i in range(len(train_bcc_val)):
  path_image = os.path.join(pp_val_directory, "bcc", train_bcc_val[i][-12:])
  image_read= cv2.imread(train_bcc_val[i])
  image_resized =  cv2.resize(image_read, dim, interpolation = cv2.INTER_AREA)
  image_hair_removed= remove_hair(image_resized, SE_list)
  image_vignette_removed= remove_black_border(image_hair_removed)
  cv2.imwrite(path_image, image_vignette_removed)
  tags_bcc_val.append(train_bcc_val[i][-12:])
  labels_bcc_val.append(1)

In [ ]:
for i in range(len(train_scc_val)):
  path_image = os.path.join(pp_val_directory, "scc", train_scc_val[i][-12:])
  image_read= cv2.imread(train_scc_val[i])
  image_resized =  cv2.resize(image_read, dim, interpolation = cv2.INTER_AREA)
  image_hair_removed= remove_hair(image_resized, SE_list)
  image_vignette_removed= remove_black_border(image_hair_removed)
  cv2.imwrite(path_image, image_vignette_removed)
  tags_scc_val.append(train_scc_val[i][-12:])
  labels_scc_val.append(2)

In [ ]:
# tags_train= tags_mel_train + tags_bcc_train + tags_scc_train
# X_train= labels_mel+labels_bcc+labels_scc

In [ ]:
tags_val= tags_mel_val + tags_bcc_val+tags_scc_val
X_test= labels_mel_val +labels_bcc_val+labels_scc_val

In [ ]:
# training_set= pd.DataFrame.from_dict({'Image Tag':tags_train, 'Image Route': train_mel+train_bcc+train_scc, 'Label': X_train})
valid_set= pd.DataFrame.from_dict({'Image Tag':tags_val, 'Image Route': train_mel_val+train_bcc_val+train_scc_val, 'Label': X_test})

In [ ]:
# X_train_data= training_set['Image Route'].to_numpy()
# y_train_data= training_set['Label'].to_numpy()
X_test_data= valid_set['Image Route'].to_numpy()
y_test_data= valid_set['Label'].to_numpy()

In [ ]:
# X_train,y_train = shuffle(X_train_data,y_train_data, random_state= 42)
X_test,y_test= shuffle(X_test_data, y_test_data, random_state=42)